In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import FreqDist , ConditionalFreqDist
from nltk.corpus import stopwords
import string
import re
from collections import defaultdict
import math

In [2]:
dataset = pd.read_csv('1-restaurant-train.csv',sep='\t',header=None)
dataset.head()
cols = [1,0]
dataset = dataset[cols]
dataset.columns = ['Review','Rating']
dataset.head()

,Review,Rating
0,Thank you thank you thank you !! I want to th...,4
1,A Humane Society store at the Biltmore? Inter...,5
2,Don't buy Nike sneakers if you want to return ...,1
3,I have to say I love most things about Sprouts...,3
4,The tire pressure light came on a day or so ag...,5


## PreProcessing

In [3]:
nb_prior = FreqDist(dataset['Rating'])
nb_prior


FreqDist({1: 6794, 2: 8662, 3: 13895, 4: 28306, 5: 24408})

In [4]:
likelihood_dict = defaultdict(list)
likelihood_dict.fromkeys(range(1,6))

bag_of_words = []
for index, row in dataset.iterrows():
    likelihood_dict[row["Rating"]] == likelihood_dict[row["Rating"]].append(row["Review"])
    
    
print(type(likelihood_dict[4]))
# for key in likelihood_dict.keys():
#     likelihood_dict[key] = re.sub('[%s]'%re.escape(string.punctuation),"",likelihood_dict[key])

# all_words = ""
#     reviews_sent = re.sub('[%s]'%re.escape(string.punctuation),"",reviews_sent)
#     return [word.lower() for word in sent.split(" ") if word not in stopwords.words('english') and len(word)>2]


<class 'list'>


In [5]:
for key in likelihood_dict.keys():
    likelihood_dict[key] = re.sub('[%s0-9]'%re.escape(string.punctuation),""," ".join(likelihood_dict[key]))

In [12]:
for key in likelihood_dict.keys():
    likelihood_dict[key] = [word.lower() for word in likelihood_dict[key]]
    


In [14]:
likelihood_dict[1]=[word for word in likelihood_dict[1] if word not in stopwords.words('english')]

In [16]:
likelihood_dict[2]=[word for word in likelihood_dict[2] if word not in stopwords.words('english')]

In [17]:
likelihood_dict[3]=[word for word in likelihood_dict[3] if word not in stopwords.words('english')]

In [18]:
likelihood_dict[4]=[word for word in likelihood_dict[4] if word not in stopwords.words('english')]

In [19]:
likelihood_dict[5]=[word for word in likelihood_dict[5] if word not in stopwords.words('english')]

In [22]:
for i in likelihood_dict.keys():
    likelihood_dict[i] = FreqDist(likelihood_dict[i])


In [32]:
len(likelihood_dict[1].keys())

109516

## Naive Bayes - Unigram Probabilites

In [66]:
def nb_model(test_sent):
    ratings = [0]
    for i in likelihood_dict.keys():
        total_len = sum(likelihood_dict[i].values())
        test_sent = re.sub('[%s0-9]'%re.escape(string.punctuation),"",test_sent)
        prob = 0
        vocab_i = len(likelihood_dict[i])
        for word in test_sent.split(" "):
            prob += math.log(likelihood_dict[i].get(word,0)+0.1)/(total_len + (0.1*vocab_i))
        total_prob = prob *nb_prior[i]/sum(nb_prior.values())
        ratings.append(total_prob)
    return ratings.index(max(ratings)),ratings

## Parts of speech tagging to get the last sentence of the review starting with noun, pronoun containing adjective 

#### as it contains more information regarding the review. This would reduce the computation needed to be done and still be more indicative of the review.

In [5]:
from nltk import pos_tag
def last_sentence_tags(s):
    s = re.sub('[%s0-9]'%re.escape(string.punctuation),"",s)
#     print(s)
    p = pos_tag([word for word in s.split(" ") if word not in stopwords.words('english') and word!=""])
#     print(p)
    for index,ele in enumerate(p[::-1]):
        if ele[1] in ("JJ","NNP","PRP"):
            
            return(p[-1*index:])
#             break


last_sentence_tags("Of course of my first visit I had to try the snacks and ended up with a chocolate and vanilla rice krispy treat, which was unusual, sorta healthy and good for post session desert. This is my kind of place, it would have been back in college too. Some things never change. Alsom oipen til midnight. Since I work at the airport I often am happy to have the chance to leave PHX occasionally and this place is unexpectedly a breath of aires fresca. It is a clean, small kitchen with a nice selection. I am always happy to support these locals and know my money is staying in town and not heading out of state when I pay my check. Watch out about the hours of operation, she'll close up early if it is slow. Good idea to  call ahead and let her know your plans.")


[('idea', 'NN'),
 ('call', 'NN'),
 ('ahead', 'RB'),
 ('let', 'VB'),
 ('know', 'VB'),
 ('plans', 'NNS')]

## Our final model - consider only the adjectives in the review and build a bayesian classifier using this information

In [7]:
from nltk import pos_tag

postag_model_dict ={}
postag_model_dict[1]=[]
postag_model_dict[2]=[]
postag_model_dict[3]=[]
postag_model_dict[4]=[]
postag_model_dict[5]=[]

for index,row in dataset[:10000].iterrows():
    if row["Rating"] :
        val = last_sentence_tags(row["Review"])
        
        postag_model_dict[row["Rating"]].extend(last_sentence_tags(row["Review"]))
#         break
    
    
    
    
# x = (dataset.loc[1]["Review"]).split(" ") 
# x = [w for w in x if w!="" and w not in string.punctuation if w not in stopwords.words('english') ]
# p = pos_tag(x)
# for index,ele in enumerate(p[::-1]):
#     if ele[1]=="DT":
#         print(index)
#         print(p[-1*index:])
#         break
postag_model_dict


{1: [('Dont', 'NNP'),
  ('buy', 'VBP'),
  ('Nike', 'NNP'),
  ('sneakers', 'NNS'),
  ('want', 'VBP'),
  ('return', 'NN'),
  ('exchange', 'NN'),
  ('Ive', 'NNP'),
  ('bought', 'VBD'),
  ('many', 'JJ'),
  ('I', 'PRP'),
  ('went', 'VBD'),
  ('week', 'NN'),
  ('bought', 'VBD'),
  ('pairs', 'NNS'),
  ('Nike', 'NNP'),
  ('SBs', 'NNP'),
  ('already', 'RB'),
  ('retail', 'JJ'),
  ('price', 'NN'),
  ('I', 'PRP'),
  ('tried', 'VBD'),
  ('one', 'CD'),
  ('side', 'NN'),
  ('bought', 'VBD'),
  ('When', 'WRB'),
  ('I', 'PRP'),
  ('got', 'VBD'),
  ('home', 'NN'),
  ('I', 'PRP'),
  ('realized', 'VBD'),
  ('one', 'CD'),
  ('shoes', 'NNS'),
  ('damaged', 'VBD'),
  ('Its', 'PRP$'),
  ('days', 'NNS'),
  ('said', 'VBD'),
  ('cant', 'JJ'),
  ('take', 'VB'),
  ('back', 'RP'),
  ('Ive', 'NNP'),
  ('bought', 'VBD'),
  ('many', 'JJ'),
  ('shoes', 'NNS'),
  ('last', 'JJ'),
  ('time', 'NN'),
  ('Im', 'NNP'),
  ('Nor', 'NNP'),
  ('Cal', 'NNP'),
  ('people', 'NNS'),
  ('crazy', 'JJ'),
  ('sneakers', 'NNS'),
  ('shop

In [8]:
##

In [11]:
for i in range(1,6):
    postag_model_dict[i] = FreqDist(postag_model_dict[i])

In [21]:
test_sentences = []
with open('1-restaurant-test.csv') as f:
    for line in f:
        
        sent =  re.sub('[%s0-9]'%re.escape(string.punctuation),"",line.strip())
        sent = " ".join([word for word in sent.split(" ") if word not in "" and stopwords.words('english')  and len(word)>2])
        test_sentences.append(sent)
        


In [88]:
# test_data_ratings = []
def predict_ratings(sent):
    tmp = [-9999]
    for key in postag_model_dict.keys():
        prob = 0
        class_len = sum(postag_model_dict[key].values())
#         print(class_len)
        vocab = len(postag_model_dict[key])
        for word in sent.split(" "):
            prob += postag_model_dict[key].get(word,1)
            pass
#                      /(class_len+vocab))
            print(prob,word,"  ",key)
        prob2 = prob * (nb_prior[key]/sum(nb_prior.values()))
        tmp.append(prob)
    return tmp.index(max(tmp)),tmp
            

In [103]:
def get_adjectives(s):
    adjectives = []
    s = re.sub('[%s0-9]'%re.escape(string.punctuation),"",s)
#     print([word for word in s.split(" ")])
    p = pos_tag([word for word in s.split(" ") if word!=""])
    for index,ele in enumerate(p):
        if ele[1] in ("JJ"):
            adjectives.append(ele[0])
    return adjectives
#             break


get_adjectives("Of course of my first visit I had to try the snacks and ended up with a chocolate and vanilla rice krispy treat, which was unusual, sorta healthy and good for post session desert. This is my kind of place, it would have been back in college too. Some things never change. Alsom oipen til midnight. Since I work at the airport I often am happy to have the chance to leave PHX occasionally and this place is unexpectedly a breath of aires fresca. It is a clean, small kitchen with a nice selection. I am always happy to support these locals and know my money is staying in town and not heading out of state when I pay my check. Watch out about the hours of operation, she'll close up early if it is slow. Good idea to  call ahead and let her know your plans.")


['first',
 'unusual',
 'healthy',
 'good',
 'oipen',
 'happy',
 'clean',
 'small',
 'nice',
 'happy',
 'slow']

In [104]:
# print(test_sentences[1])
# predict_ratings(test_sentences[0])

adjective_model_dict ={}
adjective_model_dict[1]=[]
adjective_model_dict[2]=[]
adjective_model_dict[3]=[]
adjective_model_dict[4]=[]
adjective_model_dict[5]=[]

for index,row in dataset[:10000].iterrows():
    if row["Rating"] :
        val = get_adjectives(row["Review"])
        
        adjective_model_dict[row["Rating"]].extend(val)
        
    



In [131]:
# for i in range(1,6):
#     adjective_model_dict[i] = FreqDist(adjective_model_dict[i])

# t = 0
# for i in range(1,6):
#     t += sum(adjective_model_dict[i].values())

t

v = 0
for i in range(1,6):
#     print(len(adjective_model_dict[i].keys()))
    v += len(adjective_model_dict[i].keys())
    
v

22376

In [119]:
adjective_model_dict[3].most_common(10)

[('good', 1742),
 ('great', 710),
 ('other', 586),
 ('little', 571),
 ('nice', 489),
 ('few', 373),
 ('small', 332),
 ('bad', 313),
 ('much', 309),
 ('first', 287)]

In [175]:
test_data_ratings = []
def predict_ratings(sent):
    tmp = [-9999]
    for key in adjective_model_dict.keys():
        prob = 0
        
#         print(class_len)
        words_per_key = len(adjective_model_dict[key])
        for word in sent.split(" "):
            prob += math.log(adjective_model_dict[key].get(word,1)  / (words_per_key+v) )
            
        prob2 = prob * (nb_prior[key]/sum(nb_prior.values()))
        tmp.append(prob)
    return tmp.index(max(tmp))
  


In [176]:

# predict_ratings(" ")
for sent in test_sentences:
    test_data_ratings.append((predict_ratings(sent)))

# test_data_ratings

In [185]:
output = pd.DataFrame(test_data_ratings)
output.to_csv("restaurant_results.csv")